In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os, random

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

In [2]:
all_df = pd.read_csv('../input/fda-termproject/all_asset_2010_2020.csv')
all_df['Date'] = all_df.Date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
all_df.set_index('Date',inplace=True)

In [3]:
symbols = ['AAPL', 'MSFT', 'AMZN', 'FB']#, 'GOOGL', 'GOOG', 'TSLA', 'NVDA', 'CMCSA', 'ADBE']
st ='2020-06-01'
en = '2020-09-30'

In [4]:
def EWMA(df, asset):
    '''
    *input*
    df : asset 전체 다 있는 전체 데이터 
    asset : asset symbol 하나씩
    
    *output*
    pred_price : 월 말일 price 값 딱 하나. 
                 만약2020-10-30까지가 우리가 사용할 데이터이면, 11월 30일의 price 예측 값. 
    pred_ret : 한 달 return
               만약2020-10-30까지가 우리가 사용할 데이터이면, 1
               1월 30일의 price 예측 값과 10월 30일의 실제 price를 이용해서 구한 return 
    '''
    monthly_price = df[:en][asset].resample('M').mean() # 매달 평균 값
    mean = monthly_price.iloc[-1]
    
    pred_price = monthly_price.ewm(0.9).mean().iloc[-1] # 파라미터 0.9는 대략 최근 10개 정도의 시계열 값을 평균낸 것에 근사함. 1/(1-0.9) 이런식으로 계산 가능.
    pred_ret = (pred_price - mean)/mean
    
    return pred_price, pred_ret

In [5]:
mth_pred_asset = dict()
for asset in symbols:
    
    mth_pred_asset[asset] = [EWMA(all_df, asset)]

In [6]:
mth_pred_asset

{'AAPL': [(114.30286229230443, -0.01131107924133213)],
 'MSFT': [(210.7578581613845, 0.004657629731012347)],
 'AMZN': [(3163.157885991625, 0.004582134896408934)],
 'FB': [(264.17502849474914, 0.008686653880427024)]}

In [7]:
all_df[['AAPL', 'MSFT', 'AMZN', 'FB']].iloc[-1]

AAPL     108.672516
MSFT     201.941315
AMZN    3036.149902
FB       263.109985
Name: 2020-10-30 00:00:00, dtype: float64